This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import pandas as pd


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:

key = 'PUT HERE'

In [24]:
def requests_quandl(ticker,start_date,end_date,cadence,API_KEY):
    #collect data 
    api_url = "https://www.quandl.com/api/v3/datasets/FSE/{}?start_date={}&end_date={}&collapse={}&api_key={}".format(
        ticker,start_date,end_date,cadence,API_KEY)
    data = requests.get(api_url)
    #convert to json
    data_json = json.loads(data.content)
    #convert to phd
    #print(data_json)
    return pd.DataFrame(data_json['dataset']['data'], columns = data_json['dataset']['column_names'])
    
    

In [25]:
output = requests_quandl(ticker ='AFX_X',start_date = '2017-01-01',end_date = '2017-12-31',cadence = 'daily',API_KEY = key)

In [26]:
output.head()
#data

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,None
3,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,None
4,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,None


In [27]:
# 3 highest and lowest opening prices 
print('#3 In 2017 Highest opening price in  was {}, Lowest opening price was {}'.format(output['Open'].max(),
                                                                                        output['Open'].min()))


#3 In 2017 Highest opening price in  was 53.11, Lowest opening price was 34.0


In [28]:
percent_change = ((output['High'] - output['Low'])/output['High'])*100
change_index = abs(percent_change).sort_values().index[0]
print("#4:Highest Change in one day was on {} with {}% change".format(output.iloc[change_index,0],
                                                                      percent_change.sort_values()[0]))

#4:Highest Change in one day was on 2017-09-08 with 0.9433962264150846% change


In [29]:
output['Open_shift_1day'] = output['Open'].shift(1).fillna(0)

In [30]:
percent_change_2day = ((output['Close'] - output['Open_shift_1day'])/output['Close'])*100
change_index_2day = abs(percent_change_2day).sort_values().index[0]
print("#5:Largest change between 2 days was on {} and {} with {}% change between Closing and Opening".format(
                                                                                output.iloc[change_index_2day+1,0],
                                                                                output.iloc[change_index_2day,0],
                                                                                percent_change_2day.sort_values()[0]))

#5:Largest change between 2 days was on 2017-07-25 and 2017-07-26 with 100.0% change between Closing and Opening


In [31]:
print("#6 Average trading volume for the year is {}".format(output['Traded Volume'].mean()))

#6 Average trading volume for the year is 89124.33725490196


In [32]:
print("#7 Median trading volume for the year is {}".format(output['Traded Volume'].median()))

#7 Median trading volume for the year is 76286.0
